**Compute Financial Ratios using WRDS/S&P Compustat**

In [4]:
# Import required libraries 
import pandas as pd
from finance_lib_no_wrds import calculate_ccc, get_data_from_csv

In [5]:
# Get data from csv file
comp = get_data_from_csv('sp500_data.csv')

# Filter for the stock of interest
comp = comp[comp['tic'] == 'AAPL']

# Inspect data
print(comp)

     Unnamed: 0  gvkey   tic       conm    datadate        lt        at  \
210         210   1690  AAPL  APPLE INC  2010-09-30   27392.0   75183.0   
211         211   1690  AAPL  APPLE INC  2011-09-30   39756.0  116371.0   
212         212   1690  AAPL  APPLE INC  2012-09-30   57854.0  176064.0   
213         213   1690  AAPL  APPLE INC  2013-09-30   83451.0  207000.0   
214         214   1690  AAPL  APPLE INC  2014-09-30  120292.0  231839.0   
215         215   1690  AAPL  APPLE INC  2015-09-30  171124.0  290479.0   
216         216   1690  AAPL  APPLE INC  2016-09-30  193437.0  321686.0   
217         217   1690  AAPL  APPLE INC  2017-09-30  241272.0  375319.0   
218         218   1690  AAPL  APPLE INC  2018-09-30  258578.0  365725.0   
219         219   1690  AAPL  APPLE INC  2019-09-30  248028.0  338516.0   
220         220   1690  AAPL  APPLE INC  2020-09-30  258549.0  323888.0   
221         221   1690  AAPL  APPLE INC  2021-09-30  287912.0  351002.0   
222         222   1690  A

In [6]:
# Compute year from datadate
comp['datadate'] = pd.to_datetime(comp['datadate']) #convert datadate to date fmt
comp['year'] = comp['datadate'].dt.year #get year from datadate

In [7]:
## Define Financial Metrics

# Leverage Ratios
comp['debt']=comp['dltt'] + comp['dlc'] # TOTAL DEBT = LONG TERM + SHORT TERM DEBT
comp['de_ratio'] = comp['debt'] / comp['seq'] # D/E RATIO = TOTAL DEBT / TOTAL SHAREHOLDER'S EQUITY
comp['da_ratio'] = comp['debt'] / comp['at'] # D/A RATIO = TOTAL DEBT / TOTAL ASSETS

# Liquidity Ratios 
comp['current_ratio'] = comp['act'] / comp['lct'] # CURRENT RATIO = CURRENT ASSETS / CURRENT LIABILITIES
comp['quick_ratio'] = (comp['act'] - comp['invt']) / comp['lct'] # QUICK RATIO = (CURRENT ASSETS-INVENTORY) / CURRENT LIABILITIES

# Profitability Ratios 
comp['roa'] = comp['ni'] / comp['at'] # ROA = NET INCOME / TOTAL ASSETS
comp['roe'] = comp['ni'] / comp['seq'] # ROE = NET INCOME / TOTAL SHAREHOLDER'S EQUITY
comp['gross_margin'] = (comp['sale']-comp['cogs']) / comp['sale'] # GROSS MARGIN = (SALES - COGS)/ SALES
comp['operating_margin'] = comp['oibdp'] / comp['sale'] # OPERATING MARGIN = OPERATING INCOME / SALES

# Efficiency Ratios 
comp['lag_invt'] = comp.groupby('tic')['invt'].shift() # Compute Lag Inventory 
comp['avg_invt'] = (comp['invt'] + comp['lag_invt']) / 2 # Compute Average Inventory

comp['days_in_inventory'] = comp['avg_invt'] / (comp['cogs'] / 360) # DAYS IN INVENTORY = AVERAGE INVENTORY / (COGS/360)
comp['inventory_turns'] = 360 / comp['days_in_inventory'] # INVENTORY TURNS = 360 / DAYS IN INVENTORY

comp['days_sales_outstanding'] = comp['rect'] / (comp['sale'] / 360) # DAYS SALES OUTSTANDING =  ACCOUNTS RECEIVABLE / (SALES/360)
comp['days_payable_outstanding'] = comp['ap'] / (comp['cogs'] / 360) # DAYS PAYABLE OUTSTANDING = ACCOUNTS PAYABLE / (COSGS/360)

comp['ppe_turnover'] = comp['sale'] / comp['ppent'] # PPE TURNOVER = SALES / PPE 
comp['asset_turnover'] = comp['sale'] / comp['at'] # ASSET TURNOVER = SALES / TOTAL ASSETS


In [8]:
comp['cash_conversion_cycle'] = calculate_ccc(dio=comp['days_in_inventory'], dso=comp['days_sales_outstanding'], dpo=comp['days_payable_outstanding'])

In [9]:
# View last available values
comp.sort_values('datadate').groupby('tic').tail(1)

,Unnamed: 0,gvkey,tic,conm,datadate,lt,at,lct,act,seq,...,operating_margin,lag_invt,avg_invt,days_in_inventory,inventory_turns,days_sales_outstanding,days_payable_outstanding,ppe_turnover,asset_turnover,cash_conversion_cycle
223,223,1690,AAPL,APPLE INC,2023-09-30,290437.0,352583.0,145308.0,143566.0,62146.0,...,0.320391,4946.0,5638.5,9.871084,36.470161,57.280092,109.61043,7.04879,1.087077,-42.459255
